# HW5: Ensemble methods and cross validation with college data

Using the college statistic data to compare different ensemble methods on their performance with cross validation.


In [0]:
# import few things we will use in this exercise
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier

In [2]:
#1 Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 19.8MB/s 


In [0]:
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'185gv_hzqVvSUI_TBDINYNU6Leawu5tMo'}) # replace the id with id of file you want to access
downloaded.GetContentFile('college.csv')

In [0]:
# read in the data and save it in a dataframe
col = pd.read_csv('college.csv')

In [5]:
# Take a look and see what's in it
col.sample(10)

,Unnamed: 0,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
695,University of West Florida,No,1558,1254,472,20,57,3754,2477,6172,3994,541,1387,83,87,23.4,12,8488,53
328,Loyola University,Yes,1891,1698,719,24,80,2740,761,11100,5870,600,750,77,88,11.7,14,9456,53
319,Livingstone College,Yes,900,473,217,22,47,621,11,4400,3400,800,900,53,93,10.4,16,9268,92
717,Voorhees College,Yes,1465,1006,188,10,30,703,20,4450,2522,500,1200,43,43,22.9,3,5861,58
681,University of St. Thomas MN,Yes,2057,1807,828,26,53,4106,982,11712,4037,500,800,80,80,13.8,13,8546,89
126,College of Mount St. Joseph,Yes,798,620,238,14,41,1165,1232,9800,4430,400,1150,46,46,11.1,35,6889,100
145,Concordia College at St. Paul,Yes,281,266,139,13,29,1049,181,10500,3750,450,950,69,75,12.8,18,6955,45
26,Assumption College,Yes,2135,1700,491,23,59,1708,689,12000,5920,500,500,93,93,13.8,30,7100,88
218,George Mason University,No,5653,4326,1727,17,29,9528,3822,10800,4840,580,1050,93,96,19.3,7,6751,46
104,Central Washington University,No,2785,2011,1007,8,65,6507,898,7242,3603,654,1416,67,89,18.1,0,6413,51


---
## EDA: 
##### You may perform more EDA steps than the ones suggested below.

In [6]:
# See what kind of data format and how many are there in each column
col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 19 columns):
Unnamed: 0     777 non-null object
Private        777 non-null object
Apps           777 non-null int64
Accept         777 non-null int64
Enroll         777 non-null int64
Top10perc      777 non-null int64
Top25perc      777 non-null int64
F.Undergrad    777 non-null int64
P.Undergrad    777 non-null int64
Outstate       777 non-null int64
Room.Board     777 non-null int64
Books          777 non-null int64
Personal       777 non-null int64
PhD            777 non-null int64
Terminal       777 non-null int64
S.F.Ratio      777 non-null float64
perc.alumni    777 non-null int64
Expend         777 non-null int64
Grad.Rate      777 non-null int64
dtypes: float64(1), int64(16), object(2)
memory usage: 115.4+ KB


In [7]:
# to check for any or number of null values
col.isnull().sum()

Unnamed: 0     0
Private        0
Apps           0
Accept         0
Enroll         0
Top10perc      0
Top25perc      0
F.Undergrad    0
P.Undergrad    0
Outstate       0
Room.Board     0
Books          0
Personal       0
PhD            0
Terminal       0
S.F.Ratio      0
perc.alumni    0
Expend         0
Grad.Rate      0
dtype: int64

In [8]:
# to know more about data 
col.describe()

,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
count,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000,777.00000
mean,3001.638353,2018.804376,779.972973,27.558559,55.796654,3699.907336,855.298584,10440.669241,4357.526384,549.380952,1340.642214,72.660232,79.702703,14.089704,22.743887,9660.171171,65.46332
std,3870.201484,2451.113971,929.176190,17.640364,19.804778,4850.420531,1522.431887,4023.016484,1096.696416,165.105360,677.071454,16.328155,14.722359,3.958349,12.391801,5221.768440,17.17771
min,81.000000,72.000000,35.000000,1.000000,9.000000,139.000000,1.000000,2340.000000,1780.000000,96.000000,250.000000,8.000000,24.000000,2.500000,0.000000,3186.000000,10.00000
25%,776.000000,604.000000,242.000000,15.000000,41.000000,992.000000,95.000000,7320.000000,3597.000000,470.000000,850.000000,62.000000,71.000000,11.500000,13.000000,6751.000000,53.00000
50%,1558.000000,1110.000000,434.000000,23.000000,54.000000,1707.000000,353.000000,9990.000000,4200.000000,500.000000,1200.000000,75.000000,82.000000,13.600000,21.000000,8377.000000,65.00000
75%,3624.000000,2424.000000,902.000000,35.000000,69.000000,4005.000000,967.000000,12925.000000,5050.000000,600.000000,1700.000000,85.000000,92.000000,16.500000,31.000000,10830.000000,78.00000
max,48094.000000,26330.000000,6392.000000,96.000000,100.000000,31643.000000,21836.000000,21700.000000,8124.000000,2340.000000,6800.000000,103.000000,100.000000,39.800000,64.000000,56233.000000,118.00000


---

#### Rename the first column name to University so it will be easier to relate.

In [9]:
#After renaming, check to see if the column names got changed.
col.rename(columns={"Unnamed: 0": "University"}, inplace=True)
col.columns

Index(['University', 'Private', 'Apps', 'Accept', 'Enroll', 'Top10perc',
       'Top25perc', 'F.Undergrad', 'P.Undergrad', 'Outstate', 'Room.Board',
       'Books', 'Personal', 'PhD', 'Terminal', 'S.F.Ratio', 'perc.alumni',
       'Expend', 'Grad.Rate'],
      dtype='object')

#### Change yes/no to 1/0 in column 'Priviate' to get it ready for model training.

In [10]:
#First of all, find out how many yes and no are there in the 'Private' column.

col['Private'].value_counts()

Yes    565
No     212
Name: Private, dtype: int64

In [11]:
#change yes to 1 and no to 0 in the "Private" column and view the changes after your action.
col['Private'] = col['Private'].map({'Yes' : 1, 'No' : 0})
col.sample(5)

,University,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
337,Manchester College,1,1004,802,239,23,63,909,51,10440,3850,525,1450,63,72,11.8,20,7940,64
291,Kentucky Wesleyan College,1,584,497,175,20,49,662,121,8000,4150,500,1300,57,65,11.3,32,7058,62
262,Huntingdon College,1,608,520,127,26,47,538,126,8080,3920,500,1100,63,72,11.4,9,7703,44
114,Claremont McKenna College,1,1860,767,227,71,93,887,1,17000,6010,500,850,99,99,9.6,52,18443,87
120,Coe College,1,1006,742,275,29,60,1127,205,13925,4390,500,2200,73,86,12.7,32,10141,67


In [12]:
#check to see if there are equal number of 1/0s as that in yes/no you checked earilier.
col['Private'].value_counts()

1    565
0    212
Name: Private, dtype: int64

___
### Compare classification performance of different models with cross validation

In [0]:
# Assign the Private column as target y and 
#assign all columns, except University and Private, in the dataframe as X
y = col[['Private']]
X = col.drop(['Private','University'],axis=1)

In [14]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
print(X.shape)
print(y.shape)

(777, 17)
(777, 1)


#### Create a function that takes a model and the 'number of splits' parameter in StratifiedKFold cross validation

In [0]:
def cross_score(model,model_name,n_splits, X, y):
    """
    Function that shows performance avearge and standard deviation 
    with different models and differnt number of splits in cross validation.

    Args:
        model: machine learning model used, e.g., DecisionTreeClassifier()
        model_name: The name of the machine learning model used, e.g., 'Decision Tree' Classifier.
        n_splits: number of folds used in StratifiedKFold
        X: feature data used for the model
        y: target data used for the model
    Returns:
        none
    Printout: 
        the average and the standard deviation of cross validation score 
        with 3 decimal places like the following.
        
        Decision Tree Classifier with 2 splits has an average score of 0.916 ± 0.038

    """
    ### your code here ###
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True,random_state=0)
    dt = model
    s = cross_val_score(dt, X, y, cv=cv, n_jobs=-1)
    print("{} with {} splits has an average Score of :\t{:0.3} ± {:0.3}".format(model_name, n_splits, s.mean().round(3), s.std().round(3)))

### Testing the function on Decision Tree algorithm

In [16]:
# show the  the average and the standard deviation of cross validation score 
#for splits ranging from 2 to 20

for splits in range(2,21):
  cross_score(DecisionTreeClassifier(random_state=0) ,"Decision Tree Classifier" ,splits , X, y)

Decision Tree Classifier with 2 splits has an average Score of :	0.905 ± 0.005
Decision Tree Classifier with 3 splits has an average Score of :	0.903 ± 0.02
Decision Tree Classifier with 4 splits has an average Score of :	0.897 ± 0.01
Decision Tree Classifier with 5 splits has an average Score of :	0.911 ± 0.026
Decision Tree Classifier with 6 splits has an average Score of :	0.903 ± 0.018
Decision Tree Classifier with 7 splits has an average Score of :	0.913 ± 0.023
Decision Tree Classifier with 8 splits has an average Score of :	0.921 ± 0.019
Decision Tree Classifier with 9 splits has an average Score of :	0.912 ± 0.025
Decision Tree Classifier with 10 splits has an average Score of :	0.909 ± 0.029
Decision Tree Classifier with 11 splits has an average Score of :	0.917 ± 0.035
Decision Tree Classifier with 12 splits has an average Score of :	0.914 ± 0.02
Decision Tree Classifier with 13 splits has an average Score of :	0.915 ± 0.038
Decision Tree Classifier with 14 splits has an aver

#### By examining the result above, which split(s) gives you best performance?

Ans: Split 8 gives the best performance based on random state 0.

___
### Comparing the mean and sd of cross validation scores subject to decision tree and ensemble methods

Use the function you created above to print out  
the mean and sd of cross validation scores for  
decision trees, bagging, random forest, extra trees ,
AdaBoost, and Gradient Boosting classifiers   
with 3 folds cross validation.

In [17]:
n=3
cross_score(DecisionTreeClassifier() ,"Decision Tree Classifier" ,3 , X, y)
cross_score(BaggingClassifier() ,"Bagging Classifier" ,3 , X, y)
cross_score(RandomForestClassifier() ,"Random Forest Classifier" ,3 , X, y)
cross_score(ExtraTreesClassifier() ,"Extra Trees Classifier" ,3 , X, y)
cross_score(AdaBoostClassifier() ,"AdaBoost Classifier" ,3 , X, y)
cross_score(GradientBoostingClassifier() ,"Gradient Boosting Classifier" ,3 , X, y)

Decision Tree Classifier with 3 splits has an average Score of :	0.894 ± 0.026
Bagging Classifier with 3 splits has an average Score of :	0.929 ± 0.012
Random Forest Classifier with 3 splits has an average Score of :	0.924 ± 0.012
Extra Trees Classifier with 3 splits has an average Score of :	0.936 ± 0.004
AdaBoost Classifier with 3 splits has an average Score of :	0.934 ± 0.018
Gradient Boosting Classifier with 3 splits has an average Score of :	0.941 ± 0.016


#### Comment on the mean and the sd deviation of scores and see if they are consistent with the pros and cons of ensemble methods.   
You may want to run few times for generalized comparision purposes.<br/>
**<font color='darkblue'> 
  The mean of Decision tree classifier is the lowest as expected and it has high std deviation as well so fluctuates a lot. But as we go to more complicated algorithm like extra trees and gradient boosting the mean improves but the std deviation is still more, so still it fluctuates a lot.<br/>
As the complexity increases the model gets better by increasing the score but has the problem of overfitting due to increase in complexity.
  Yes, they are consistent with the pros and cons of emsemble methods. </font> **

---

### Print out the "feature importances" of this best model.

The model has an attribute called `.feature_importances_` which can tell us which features were most important vs. others. It ranges from 0 to 1, with 1 being the most important.

An easy way to think about the feature importance is how much that particular variable was used to make decisions. Really though, it also takes into account how much that feature contributed to splitting up the class or reducing the variance.

A feature with higher feature importance reduced the criterion (impurity) more than the other features.

See 
<a href="https://towardsdatascience.com/running-random-forests-inspect-the-feature-importances-with-this-code-2b00dd72b92e">here </a> for more detail and coding sample.

#### Show, for RandomForestClassifier, the feature importances for each variable predicting private vs. not, sorted by most important feature to least.

In [18]:
# show the ranked list of feature importances here
from sklearn.model_selection import train_test_split
import pandas as pd 
rf = RandomForestClassifier()
rf.fit(X,y)
rf.score(X,y)
feature_importances = pd.DataFrame(rf.feature_importances_, index = X.columns, columns=['Importance']).sort_values('Importance', ascending = False)
feature_importances

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


,Importance
Outstate,0.232804
F.Undergrad,0.202376
Enroll,0.106723
Room.Board,0.073403
P.Undergrad,0.068718
Accept,0.066836
Terminal,0.046625
S.F.Ratio,0.045178
Expend,0.024570
Top10perc,0.022684


#### Show a ranked list of features in col dataframe that is correlated to the target column 'Private'

In [19]:
# ranked list of features in col dataframe that is correlated to the target column 'Private'
col.corr().abs()['Private'].sort_values(ascending = False)[1:]

F.Undergrad    0.615561
Enroll         0.567908
Outstate       0.552650
Accept         0.475252
S.F.Ratio      0.472205
P.Undergrad    0.452088
Apps           0.432095
perc.alumni    0.414775
Room.Board     0.340532
Grad.Rate      0.336162
Personal       0.304485
Expend         0.258461
Top10perc      0.164132
PhD            0.156714
Terminal       0.129620
Top25perc      0.095752
Books          0.018549
Name: Private, dtype: float64

#### Compare and comment on the feature importances and corr() lists you have above.

Provide your observation and interpretation here.<br/>
**<font color='darkblue'>
  There is lot of difference in the correlation and feature importance score , As I see correlation gives much bigger number when compared to feature importance. According to my opinion, feature importance is determined after the model is fitted on data and correlation is just done on prior available raw data linearly, so the feature importance is considered more accurate interpretation of relationship between features and target and is specific to the algorithm used but the correlation is a more generalized version.
</font>**